In [7]:
from z3 import *
import numpy as np

def find_6bit_stabilizer_shapes(H=5, L=11, d=9, weight_limit=3):
    s = Solver()
    num_rows = H - 2  # number of stabilizer rows
    b = [[Bool(f"b{j}_{i}") for j in range(6)] for i in range(num_rows)]  # 6-bit shape per row

    # Constraint: at most 'weight_limit' 1's per stabilizer
    for i in range(num_rows):
        s.add(Sum([If(b[i][j], 1, 0) for j in range(6)]) <= weight_limit)

    # Brute-force all 2-row initializations
    for init_val in range(1, 2 ** (2 * L)):
        bits = [bool((init_val >> i) & 1) for i in range(2 * L)]
        Codeword = [BoolVal(bit) for bit in bits]

        # Fill codeword based on stabilizer shape
        for row in range(2, H):
            for col in range(L):
                shape = b[row - 2]

                def shift(idx):
                    return (col + idx + L) % L  # wrap around

                ml = Codeword[(row - 1) * L + shift(-1)]  # bit 0
                mc = Codeword[(row - 1) * L + shift( 0)]  # bit 1
                mr = Codeword[(row - 1) * L + shift(+1)]  # bit 2
                tl = Codeword[(row - 2) * L + shift(-1)]  # bit 3
                tc = Codeword[(row - 2) * L + shift( 0)]  # bit 4
                tr = Codeword[(row - 2) * L + shift(+1)]  # bit 5

                new_bit = Xor(
                    Xor(ml & shape[0], mc & shape[1]),
                    Xor(mr & shape[2],
                        Xor(tl & shape[3], tc & shape[4] ^ (tr & shape[5]))
                    )
                )

                Codeword.append(new_bit)

        s.add(Sum([If(bit, 1, 0) for bit in Codeword]) >= d)

    if s.check() == sat:
        model = s.model()
        print("SAT: Valid stabilizer shapes found!\n")
        for i in range(num_rows):
            bits = [int(is_true(model.evaluate(b[i][j]))) for j in range(6)]
            # Map to 3x3 matrix for visualization
            shape = np.array([
                [0, 1, 0],  # bottom row (n)
                [bits[0], bits[1], bits[2]],  # middle row (n−1)
                [bits[3], bits[4], bits[5]]  # top row (n−2)
            ])
            print(f"Shape {i}:\n{shape}\n")
        return model
    else:
        print("UNSAT: No shape found meeting constraints.")
        return None

# Run it
find_6bit_stabilizer_shapes(H=4, L=5, d=5, weight_limit=3)


SAT: Valid stabilizer shapes found!

Shape 0:
[[0 1 0]
 [0 1 0]
 [1 0 1]]

Shape 1:
[[0 1 0]
 [1 0 1]
 [0 1 0]]



[b2_0 = False,
 b3_1 = False,
 b5_0 = True,
 b4_0 = False,
 b0_1 = True,
 b0_0 = False,
 b3_0 = True,
 b2_1 = True,
 b4_1 = True,
 b1_0 = True,
 b5_1 = False,
 b1_1 = False]